Author:
        
        PARK, JunHo, junho@ccnets.org

        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

In [ ]:
import sys

path_append = "../" # Go up one directory from where you are.
sys.path.append(path_append) 
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from nn.utils.init import set_random_seed
set_random_seed(0)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import torch
import torchvision.datasets as dset
from torchvision import transforms

# import albumentations
n_img_sz = 128
# Load the CelebA dataset for training. Specify the root directory where the dataset is located
trainset = dset.CelebA(root=path_append + '../data/celeba', split = "train", transform=transforms.Compose([
                            transforms.Resize(n_img_sz), # Transformations include resizing the images to `n_img_sz`
                            transforms.CenterCrop(n_img_sz), # Center cropping to the same size
                            transforms.ToTensor(), # Converting the images to tensors,
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Normalizing the pixel values to have a mean and standard deviation of 0.5 across all channels.
                        ]), download= False)

testset = dset.CelebA(root=path_append + '../data/celeba', split = "test", transform=transforms.Compose([
                            transforms.Resize(n_img_sz), # Transformations include resizing the images to `n_img_sz`
                            transforms.CenterCrop(n_img_sz), # Center cropping to the same size
                            transforms.ToTensor(), # Converting the images to tensors
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Normalizing the pixel values to have a mean and standard deviation of 0.5 across all channels.
                        ]), download= False)    


In [ ]:
label_list = ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 
              'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 
              'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 
              'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 
              'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young' ]

causal_model_selected_attribute = torch.tensor([label_list.index('Male')])
causal_model_none_selected_attribute = torch.tensor([label_list.index('Eyeglasses')])

In [ ]:
# Custom dataset class for CelebA dataset
class CelebADataset(Dataset):
    def __init__(self, dataset, selected_attributes):
        self.dataset = dataset
        self.selected_attributes = selected_attributes
    
    def __getitem__(self, index):
        X, y = self.dataset[index]
        y = torch.index_select(y.unsqueeze(0), 1, self.selected_attributes).squeeze(0)
        return X, y
    
    def __len__(self):
        return len(self.dataset)
    
class ExplainDataset(Dataset):
    def __init__(self, explanations, labels):
        self.explanations = explanations
        self.labels = labels
    
    def __getitem__(self, index):
        return self.explanations[index], self.labels[index]
    
    def __len__(self):
        return len(self.explanations)

In [ ]:
causal_model_dataset = CausalModelDataset(trainset, causal_model_labels)

In [ ]:
from trainer_hub import TrainerHub
num_classes = len(causal_model_selected_labels) 
data_config = DataConfig(dataset_name = 'celebA', task_type='multi_label_classification', obs_shape=[3, n_img_sz, n_img_sz], \
                        label_size=num_classes, show_image_indices=[737, 1518, 390, 607])

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'resnet')

ml_params.training.num_epoch = 1
# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device)

In [ ]:
trainer_hub.train(causal_model_dataset)    

In [ ]:
def generate_encoding_dataset(dataset, selected_attribute):
    data_loader = DataLoader(dataset=dataset, batch_size=64, shuffle=False, drop_last=False)

    encoding_list = []
    attribute_list = []

    causal_model = trainer_hub.ccnet

    for batch_idx, (images, attributes) in enumerate(data_loader):
        encoding = causal_model.explain(images)
        encoding_list.append(encoding)
        # Extract specified selected_attributes using indices
        attribute_list.append(attributes[:, selected_attribute])

    # Concatenate all collected causal_encodeds and selected_attributes into tensors
    encoding_size = causal_encoding_tensor.size(-1)

    causal_encoding_tensor = torch.cat(causal_encoding_list, dim=0)
    selected_attribute_tensor = torch.cat(selected_attribute_list, dim=0)

    # Create datasets
    encoding_dataset_with_attributes = ExplainDataset(causal_encoding_tensor, selected_attribute_tensor)
    
    return encoding_dataset_with_attributes


In [ ]:
encoding_dataset_with_selected_attributes = generate_encoding_dataset(trainset, causal_model_selected_attribute)
encoding_dataset_with_none_selected_attributes = generate_encoding_dataset(trainset, causal_model_none_selected_attribute)

# Optional: DataLoader for the created datasets
encoding_loader_with_selected_attributes = DataLoader(encoding_dataset_with_selected_attributes, batch_size=64, shuffle=True)
encoding_loader_with_none_selected_attributes = DataLoader(encoding_dataset_with_none_selected_attributes, batch_size=64, shuffle=True)


In [ ]:
class AttributeClassifier(torch.nn.Module):
    def __init__(self, input_size, output_size, num_layers=4, hidden_size=256):
        super(AttributeClassifier, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Create a list to hold all layers
        layers = []
        
        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_size))
        layers.append(torch.nn.ReLU())
        
        # Hidden layers
        for _ in range(num_layers - 2):
            layers.append(torch.nn.Linear(hidden_size, hidden_size))
            layers.append(torch.nn.ReLU())
        
        # Output layer
        layers.append(torch.nn.Linear(hidden_size, output_size))
        
        # Register all layers
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return torch.sigmoid(x)

In [ ]:
# Function to train classifier
def train_classifier(model, dataset, num_epoch=5, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
    
    for epoch in range(num_epoch):
        model.train()
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(outputs, labels.unsqueeze(1).float())
            loss.backward()
            optimizer.step()
        
        print(f'\nEpoch {epoch + 1} / {num_epoch}')
        evaluate_model(model, train_loader, device)


In [ ]:
# Function to evaluate classifier
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, labels in dataloader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            preds = torch.sigmoid(outputs).round()
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
    
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Accuracy: {accuracy:.4f}')
    print(classification_report(all_labels, all_preds))
    print(confusion_matrix(all_labels, all_preds))

In [ ]:
# Train and evaluate classifiers on the explanation datasets
causal_classifier = AttributeClassifier(encoding_size, num_classes)
none_selected_classifier = AttributeClassifier(encoding_size, num_classes)

print("Training causal classifier on selected attributes...")
train_classifier(causal_classifier, encoding_dataset_with_selected_attributes, num_epoch=5, device=device)

print("Training classifier on none selected attributes...")
train_classifier(none_selected_classifier, encoding_dataset_with_none_selected_attributes, num_epoch=5, device=device)